## Load data

In [2]:
with open('wizard_of_oz.txt', 'r', encoding="utf-8") as f:
    text = f.read()

print(text[:100])

The Wonderful Wizard of Oz




Chapter I
The Cyclone


Dorothy lived in the midst of the great Kans


## Create a basic character-level tokenizer 

In [3]:
# Check the info of unique tokens in the text

chars = sorted(set(text))
print(chars)
print(f"No of unique chars in the entire text: {len(chars)}")

['\n', ' ', '!', '(', ')', '*', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '\ufeff']
No of unique chars in the entire text: 70


In [12]:
# Create basic encoder-decoder

string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# Test encoding-decoding
sample_encoding = encode("hello")
print(sample_encoding)

sample_decoding = decode(sample_encoding)
print(sample_decoding)

[45, 42, 49, 49, 52]
hello
